In this section, we’ll explore the model part of the YAML configuration file. This configuration defines how the neural network model is structured, including its layers and any pretrained weights.

Here’s the model section from YAML file:
```
model:
  pretrained_weights: "path_to_best_model.pt"
  input_layer:
    type: 'CONV_GNN'
    params:
      node_features: 84
      embedding_size_reduced: 128
  body_layer:
    type: 'SKIPBLOCK_BODY'
    params:
      embedding_size_gnn: 128
      embedding_size: 256
      num_skipblocks: 7
      pooling_fn: 'mean'
  head_layer:
    type: 'BidirectionalHeadLayer'
    params:
      input_size: 256
      output_size: 1024
```
#### Overview

	•	pretrained_weights: Path to the pretrained model weights. This is optional and used when you want to continue training or fine-tune an existing model.
	•	input_layer: Defines the type and parameters of the input layer of the model.
	•	body_layer: Defines the type and parameters of the body layer(s) of the model.
	•	head_layer: Defines the type and parameters of the head layer of the model. The head layer can be omitted if not needed.

#### Model Architecture

The model is partitioned into three main sections:

	1.	Input Layer
	2.	Body Layer
	3.	Head Layer

This modular design allows different architectures by simply changing the configuration.

#### 1. Input Layer

	•	Purpose: Processes the input data (graph representations of molecules) and extracts initial features.
	•	Defined in: mol2dreams/model/InputLayer.py

Available Input Layers

	•	CONV_GNN: Uses Graph Convolutional Networks (GCN) to process node features.
	•	TRANSFORMER_CONV: Utilizes Transformer convolutional layers.
	•	GAT: Graph Attention Networks for capturing complex relationships.
	•	NEIMS: Placeholder for molecular fingerprints (pass-through, not used).

Configuration Example
```
input_layer:
  type: 'CONV_GNN'
  params:
    node_features: 84
    embedding_size_reduced: 128
```
	•	type: Specifies the input layer class to use.
	•	params: Parameters specific to the chosen input layer (serach in code for model subpackage, from example down, we use set only node_features and embedding_size_reduced).

Code Snippet for CONV_GNN
```
class CONV_GNN(InputLayer):
    def __init__(self, node_features, embedding_size_reduced):
        super(CONV_GNN, self).__init__()
        torch.manual_seed(42)
```
#### 2. Body Layer

	•	Purpose: Acts as the core of the model, performing deep feature extraction and transformation.
	•	Defined in: mol2dreams/model/BodyLayer.py

Available Body Layers

	•	SKIPBLOCK_BODY: Consists of multiple skip blocks, which help in training deeper networks by mitigating the vanishing gradient problem.
	•	Regression_BODY: A fully connected network suitable for regression tasks.

Configuration Example
```
body_layer:
  type: 'SKIPBLOCK_BODY'
  params:
    embedding_size_gnn: 128
    embedding_size: 256
    num_skipblocks: 7
    pooling_fn: 'mean'
```
	•	type: Specifies the body layer class to use.
	•	params: Parameters specific to the chosen body layer.

Parameters Explained

	•	embedding_size_gnn: The size of the embeddings from the input layer.
	•	embedding_size: The size of the embeddings within the body layer.
	•	num_skipblocks: Number of skip blocks in the body layer.
	•	pooling_fn: The pooling function to use from GNN input layer (mean, max, or add).

Code Snippet for SKIPBLOCK_BODY
```
class SKIPBLOCK_BODY(BodyLayer):
    def __init__(self, embedding_size_gnn, embedding_size, num_skipblocks=7, pooling_fn='mean'):
        super(SKIPBLOCK_BODY, self).__init__()
        torch.manual_seed(42)
```

#### 3. Head Layer

	•	Purpose: Transforms the output of the body layer to the desired output size or performs final computations.
	•	Defined in: mol2dreams/model/HeadLayer.py
	•	Note: The head layer can be omitted if not needed; an identity layer (IdentityHead) will be used by default.

Available Head Layers

	•	BidirectionalHeadLayer: Combines forward and backward predictions using gating mechanisms.
	•	IdentityHead: Pass-through layer that outputs the input as-is.

Configuration Example
```
head_layer:
  type: 'BidirectionalHeadLayer'
  params:
    input_size: 256
    output_size: 1024
```
	•	type: Specifies the head layer class to use.
	•	params: Parameters specific to the chosen head layer.

Code Snippet for BidirectionalHeadLayer
```
class BidirectionalHeadLayer(HeadLayer):
    def __init__(self, input_size, output_size):
        super(BidirectionalHeadLayer, self).__init__()
        torch.manual_seed(42)

```
#### Model Assembly

The model is assembled by combining the input, body, and head layers.

Mol2DreaMS Class

	•	Defined in: mol2dreams/model/mol2dreams.py
	•	Purpose: Wraps the input, body, and head layers into a single model.

Code Snippet
```
class Mol2DreaMS(nn.Module):
    def __init__(self, input_layer: nn.Module, body_layer: nn.Module, head_layer: nn.Module = None):
        super(Mol2DreaMS, self).__init__()

```
Building the Model from Configuration

The model is built using the build_model_from_config function, which parses the YAML configuration and constructs the model accordingly.

#### Using the Configuration

When setting up your YAML configuration, you can customize your model architecture by:

	•	Selecting Different Layer Types:
	•	Change the type field under input_layer, body_layer, or head_layer to use different implementations.
	•	Adjusting Parameters:
	•	Modify the params dictionaries to adjust hyperparameters like embedding sizes, number of layers, etc.
	•	Including Pretrained Weights:
	•	Specify the pretrained_weights path if you want to continue training or fine-tune an existing model.

Example: Creating a Model from Scratch

If you don’t want to use pretrained weights, you can omit the pretrained_weights field:
```
model:
  input_layer:
    type: 'CONV_GNN'
    params:
      node_features: 84
      embedding_size_reduced: 128
  body_layer:
    type: 'SKIPBLOCK_BODY'
    params:
      embedding_size_gnn: 128
      embedding_size: 256
      num_skipblocks: 7
      pooling_fn: 'mean'
  head_layer:
    type: 'BidirectionalHeadLayer'
    params:
      input_size: 256
      output_size: 1024
```
Extending the Model

You can extend the model by:

	•	Adding New Layer Types:
	•	Implement new classes in the respective modules (InputLayer.py, BodyLayer.py, HeadLayer.py).
	•	Update the configuration (yaml) to use your new layer types.
	•	Customizing Existing Layers:
	•	Modify existing layer classes to adjust their behavior.
	•	Update parameters in the configuration as needed.
